# Lesson 5. Practice with SimPy

### SA421 Fall 2015

#### Problem.

The Bank of Poisson has two service counters, each with its own queue. Counter A is operated by Ann, who is very competent: service times at counter A are uniformly distributed between 1 and 3 minutes. Counter B is operated by Bob, who is new at the job: service times at counter B are uniformly distributed between 2 and 8 minutes. 

We consider the bank at its busiest, from 10:00 to 14:00. During this time period, the interarrival time between customers is exponentially distributed with a mean of 2.5 minutes. As is typical in real life, an arriving customer will always join the shorter queue.
If the queues are equal in length, assume that an arriving customer joins the queue for counter B.

1. Simulate the operation of the bank for these 4 hours. Assume that the bank is empty at 10:00.
2. What is the time average number of customers in each queue?
3. What fraction of time is Ann busy? Bob?
4. What fraction of the customers end up waiting in line for Counter A?

#### Hints.

* `resource.waitQ` is a list consisting of the processes (entities) waiting to receive a unit of resource `resource`.


* As a result, you can determine the current length of the queue of resource `resource` with `len(resource.waitQ)`.

Here's some Python setup to help you get started.

In [1]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential

# Import step plotting and histogram functions from Matplotlib
from matplotlib.pyplot import step, hist

# Run Matplotlib magic to show plots directly in the notebook
%matplotlib inline

# Make Matplotlib plots display as SVG files, which are cleaner
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

Here is the simulation model.

Both resources (`R.Ann` and `R.Bob`) have monitors activated. In addition, there is one custom-made monitor, `M.counterA`, that equals 1 if the customer chooses Counter A, and 0 if the customer chooses Counter B.

In [2]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 2.5
    interarrivalTimeMean = 2.5
    
    # Service times for Counter A are uniformly distributed 
    # between 1 and 3
    AServiceTimeMin = 1
    AServiceTimeMax = 3
    
    # Service times for Counter B are uniformly distributed 
    # between 2 and 8
    BServiceTimeMin = 2
    BServiceTimeMax = 8
    
    # Simulation period is 4 continuous hours
    simulationTimeMax =  4 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives
        print("Time {1}: {0} arrives".format(self.name, now()))
        
        # Customer looks at queue lengths, chooses shorter queue
        # If Ann's queue is shorter, then customer chooses Counter A
        if len(R.Ann.waitQ) < len(R.Bob.waitQ):        #1
            # Customer chooses Counter A
            print("Time {1}: {0} chooses Counter A".format(self.name, now()))
            M.counterA.observe(1)
            yield request, self, R.Ann
            
            # Customer is released from queue and starts service
            print("Time {1}: {0} begins service".format(self.name, now()))
            serviceTime = uniform(low = P.AServiceTimeMin, high = P.AServiceTimeMax)  
            yield hold, self, serviceTime

            # Customer finishes service, leaves
            print("Time {1}: {0} ends service and leaves".format(self.name, now()))
            yield release, self, R.Ann
            
        # Otherwise, the customer chooses Counter B
        else:
            # Customer chooses Counter B
            print("Time {1}: {0} chooses Counter B".format(self.name, now()))
            M.counterA.observe(0)
            yield request, self, R.Bob
            
            # Customer is released from queue and starts service
            print("Time {1}: {0} begins service".format(self.name, now()))
            serviceTime = uniform(low = P.BServiceTimeMin, high = P.BServiceTimeMax)
            yield hold, self, serviceTime

            # Customer finishes service, leaves
            print("Time {1}: {0} ends service and leaves".format(self.name, now()))
            yield release, self, R.Bob        

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    Ann = None
    Bob = None
    
    
##### Monitors #####
class M:
    counterA = None

    
##### Experiment #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random sampling functions
    seed(123)

    # Create resources Ann and Bob
    R.Ann = Resource(capacity = 1, monitored = True)
    R.Bob = Resource(capacity = 1, monitored = True)
    
    # Create monitor for Counter A
    M.counterA = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

Next, we run the simulation.

In [3]:
model()

Time 2.9806803587: Customer 0 arrives
Time 2.9806803587: Customer 0 chooses Counter B
Time 2.9806803587: Customer 0 begins service
Time 3.82334906539: Customer 1 arrives
Time 3.82334906539: Customer 1 chooses Counter B
Time 5.82693326993: Customer 2 arrives
Time 5.82693326993: Customer 2 chooses Counter A
Time 5.82693326993: Customer 2 begins service
Time 6.34178908009: Customer 0 ends service and leaves
Time 6.34178908009: Customer 1 begins service
Time 7.67314619018: Customer 2 ends service and leaves
Time 9.00461060865: Customer 3 arrives
Time 9.00461060865: Customer 3 chooses Counter B
Time 11.891216293: Customer 4 arrives
Time 11.891216293: Customer 4 chooses Counter A
Time 11.891216293: Customer 4 begins service
Time 13.5305167766: Customer 5 arrives
Time 13.5305167766: Customer 5 chooses Counter A
Time 13.6754513294: Customer 4 ends service and leaves
Time 13.6754513294: Customer 5 begins service
Time 14.2263742704: Customer 1 ends service and leaves
Time 14.2263742704: Customer

To compute the time average number of customers in each queue:

In [4]:
print("Time average number of customers in Counter A queue = {0}".format(R.Ann.waitMon.timeAverage()))
print("Time average number of customers in Counter B queue = {0}".format(R.Bob.waitMon.timeAverage()))

Time average number of customers in Counter A queue = 0.149484747096
Time average number of customers in Counter B queue = 0.780007337491


To compute Ann and Bob's utilization:

In [5]:
print("Counter A utilization = {0}".format(R.Ann.actMon.timeAverage()))
print("Counter B utilization = {0}".format(R.Bob.actMon.timeAverage()))

Counter A utilization = 0.468051258936
Counter B utilization = 0.965732941402


To compute the fraction of customers who join the queue for Counter A:

In [6]:
print("Fraction of customers who join the queue for Counter A = {0}".format(M.counterA.mean()))

Fraction of customers who join the queue for Counter A = 0.558823529412


Note that the simulation above assumes that when choosing a counter, arriving customers only consider the number of customers waiting in each queue. That is, these arriving customers do **not** consider a customer being served as "in the queue". Depending on your perspective, this might not seem very realistic.

How can you change the above simulation so that arriving customers also consider a customer being served as "in the queue"?

*Hint.* Look for `activeQ` in the SimPy documentation.